# [Tips of Data analysis](https://drive.google.com/file/d/12faqaslFIF-Sg_sU3jeGyauW5ClRqS8D/view)

In [1]:
import pandas as pd
import numpy as np

file_path = ""

# Change data type for reduce memory
___

In [ ]:
"""
각 컬럼 내 데이터의 최소, 최대 범위를 계산해
적절한 Data type을 찾아내는 함수

각 컬럼의 데이터 형식을 모를 때 자동으로
체크하여 사용하고 데이터를 불러올 때
체크된 데이터 형식으로 데이터를 불러옵니다

형식을 지정하지 않으면 가장 메모리를 많이 차지하는
방식으로 데이터를 불러오기 때문에
형식을 지정해서 데이터를 불러오면 메모리를 줄여
큰 데이터도 불러올 수 있습니다
"""

In [ ]:
"""
데이터 분석을 하다 보면 처리된 데이터를 따로 저장하거나 다른 사람에게 공유해야 될
때가 많습니다

일반적으로 사용하는 CSV 포맷의 단점

- string 기반으로 IO 효율이 떨어짐
- Meta data가 없어 각 컬럼의 데이터 형식 등에 대해 연속성을 가지고 사용 불가

최근 많이 사용되고 있는 hdf5, parquet 포맷과 pickle, feather 같은 다른 포맷과
1) 데이터를 읽고 쓰는 시간, 2) 메모리 사용량, 3) 저장된 파일의 크기를 비교할 것입니다.
"""

In [2]:
def check_dtypes(file_path):
    print(file_path)
    tmp = pd.read_csv(file_path, nrows=0)
    col_dtypes = {}
    for col in tmp.columns:
        df = pd.read_csv(file_path, usecols=[col])
        dtype = df[col].dtype
        
        if dtype == 'int' or dtype == 'float':
            c_min = df[col].min()
            c_max = df[col].max()
        elif dtype == 'object':
            n_unique = df[col].nunique()
            threshold = n_unique / df.shape[0]
            
        if dtype == 'int':
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                col_dtype = 'int8'
            elif c_min > np.iinfo(np.uint8).min and c_max < np.iinfo(np.uint8).max:
                col_dtype = 'uint8'
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                col_dtype = 'int16'
            elif c_min > np.iinfo(np.uint16).min and c_max < np.iinfo(np.uint16).max:
                col_dtype = 'uint16'
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                col_dtype = 'int32'    
            elif c_min > np.iinfo(np.uint32).min and c_max < np.iinfo(np.uint32).max:
                col_dtype = 'uint32'
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                col_dtype = 'int64' 
            elif c_min > np.iinfo(np.uint64).min and c_max < np.iinfo(np.uint64).max:
                col_dtype = 'uint64'
                
        elif dtype == 'float':
#            # ERROR occured When using float32 in feather, parquet
#            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                col_dtype = 'float16'
            if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                col_dtype = 'float32'
            else:
                col_dtype = 'float64'
        
        elif dtype == 'object':
            if threshold > 0.7:
                col_dtype = 'object'
            else:
                col_dtype = 'category'
        
        col_dtypes[col] = col_dtype
    
    return col_dtypes

# Vectorization
_____


In [37]:
def scoring_health(patient):
    
    bmi = ((patient['Weight'] / (patient['Height']/100**2) >= 30)) * 1
    waist = (patient['WAIST'] >= 90) * 1
    blds = (patient['BLDS'] >= 125) * 1
    chole = (patient['TOT_CHOLE'] >= 130) * 1
    trigly = (patient['TRIGLYCERIDE'] >= 150) * 1
    hmg = (patient['HMG'] < 12) * 1
    crea = (patient['CREATININE'] > 1.7) * 1
    sg = ((patient['SGOT_AST'] >= 40) | (patient['SGPT_ALT'] >= 40)) * 1
    smoke = (patient['SMOKE'] == 3) * 1
    drink = (patient['DRINK'] == 1) * 1

    patient_score = np.sum([bmi, waist, blds, trigly, hmg, crea, sg, smoke, drink], axis=0)

    return patient_score

In [ ]:
# Iteration by indexing with len(df)

"""
scores = []
for i in range(len(df)):
patient = df.iloc[i]
scores.append(scoring_health(patient))
"""

In [41]:
# Iteration by using .iterrows()

"""
scores = []
for patient in df.iterrows():
scores.append(scoring_health(patient))
"""

In [42]:
# Use Pandas.apply()

"""
scores = df.apply(scoring_health, axis=1)
"""

In [ ]:
# Vectorization with Pandas Series

"""
scores = scoring_health(df)
"""

In [ ]:
# Vectorization with Numpy Array

"""
scores = scoring_health_np(df)
"""

# Sorting
_____

In [ ]:
"""
tmp = t20['EDEC_TRAMT'].sort_values(ascending=False).head(5)

tmp = t20["EDEC_TRAMT"].nlargest(5)
"""

# Indexing
___

In [ ]:
# List Comprehension

"""
df = df[[x in patients for x in df[‘PATIENT_ID’]]]
"""

In [ ]:
# pd.Series.apply()

"""
df = df[df['PATIENT_ID'].apply(lambda x: x in patients)
"""

In [ ]:
# pd.DataFrame.isin()

"""
df = df[df.isin({‘PATIENT_ID’: patients})[‘PATIENT_ID’]]
"""

In [ ]:
# pd.DataFrame.query()

"""
df = df.query(‘PATIENT_ID in @patients’)
"""

In [ ]:
# np.isin()

"""
df = df[np.isin(df[‘PATIENT_ID’], patients)]
"""

In [ ]:
# pd.DataFrame.merge()

"""
df = df.merge(patients, how=‘inner’, on=‘PATIENT_ID’)
"""

# Inplace
___


* “inplace” parameter는 Pandas DataFrame에서 작업을 하다가 결과를 바로 해당
DataFrame에 덮어씌우고 싶을 때 많이 사용합니다.


### inplace를 왜 사용하는가, 왜 사용하지 말아야 하는가?

inplace를 선호하는 사용자들의 여러 의견은 다음과 같습니다

- 속도가 더 빠르다

- 메모리를 더 효율적으로 사용한다

이에 Pandas Core 개발자들은 반대를 하고 있으며,
이미 inplace parameter에 대한 deprecation 및 삭제를 논의하고 있고,
Method Chaining 방식을 적극 권장하고 있습니다

inplace 실행 이후에도 메모리에 데이터가 남아있는 문제가 자주 있기 때문입니다
___

### 어떤식으로 구분해서 사용하는 것이 좋은가

a. Method Chaining

- 결과 DataFrame의 전체를 생성하고 재할당(reassign)하는 특징
- chaining 과정에서 데이터가 크기가 줄어들 때 메모리 효율적
- 따라서 .drop(), .astype() 등을 우선적으로 사용하는 것이 좋음

b. inplace parameter

- 추상화된 함수 내부에서 DataFrame의 일부만 생성되어 재할당(reassign)되는 특징
- 큰 작업 단위에서 Method Changing 보다 성능, 메모리 사용에서 종종 우위를 보임
- 따라서 .set_index(), .rename() 과 같이 일부 내용만 변경하는 경우에 효율적

___

# How Scaling?
1. Buy more RAM...
2. Other Dataframe frameworks

- Dask, Modin

3. GPU

- Numba, cuDF

4. Parallelization module

- multiprocessing
5. DBMS, Spark ...